In [179]:
import pandas as pd
import numpy as np
import openpyxl

In [181]:
def get_sections(file_names):
    """ Get all the sections within a particular group.
    
    Args:
        file_names (list): has the names of all the csv files (that look like term test *.csv) from which all section names will
                       be taken out
    
    Returns:
        sections (list): list of all the sections within the particular gorup.
    """
    df = pd.read_csv(file_names[0])
    sections = df['Section'].unique()
    return sections

In [182]:
def remove_unwanted_cols(file_names):
    """ Gets rid of all the unecessary columns that one does not want in the final gradebook
    
    Args:
        file_names(list): has the names of all the csv files (that look like term test *.csv) from which all section names will
                          be taken out
                       
    Returns:
        list_of_dfs (list): a list of dataframes with all the unwanted columns removed from each of the dataframes
    
    """
    list_of_dfs = []
    for file_name in file_names:
        df = pd.read_csv(file_name)
        remove_cols = ['Crowdmark ID', 'Score URL', 'MC-total', 'Total', 'Penalty','Custom Penalty', 'Submitted At', 'State']
        # get the question columns
        q_cols = [col for col in df.columns if col[0] == 'Q']
        # include the question columns in the remove_cols
        remove_cols.extend(q_cols)
        # remove the unwanted columns
        df.drop(remove_cols, axis = 1, inplace=True)
        # append the dataframe containing the required columns to the list of dataframes
        list_of_dfs.append(df)
    return list_of_dfs

In [183]:
def section_only_results(list_of_dfs, section):
    """ Gets the list of dataframes for a particular section
    
    Args:
        list_of_dfs (list): the list of dataframes (returned by remove_unwanted_cols(file_names) function)
        section (str): section name for which you want the dataframes
    
    Returns:
        clean_dfs (list): list of dataframes wherein each dataframe has results corresponding only to the section provided 
                          as an arugment to the function
    
    """
    clean_dfs = []
    for index, df in enumerate(list_of_dfs):
        clean_df = df[df['Section'] == section]
        num_entries = clean_df.shape[0]
        
        indices = [i for i in range(1,num_entries+1)]
        clean_df.index = indices
        
        # rename columns
        clean_df.rename(columns = {'Student ID Number':'Student ID', 'Total After Penalty':f'Term Test {index +1}'}, inplace = True)
        
        clean_dfs.append(clean_df)
    return clean_dfs

In [184]:
def sort_dfs(clean_dfs):
    sorted_dfs = []
    for clean_df in clean_dfs:
        sorted_df = clean_df.sort_values('Name')
        sorted_df.set_index(['Email', 'Name', 'Student ID', 'Section'], inplace = True)
        sorted_dfs.append(sorted_df)
    return sorted_dfs

In [185]:
def get_final_dataframe(sorted_dfs):
    """ Returns a single dataframe that contains results of all the term tests (merged from all the dataframes in clean_dfs)
    
    Args:
        sorted_dfs (list): the list of sorted dataframes returned by sort_dfs(clean_dfs) function
    
    Returns:
        final_df (dataframe): the dataframe consists of results from all the term tests for each of teh student
    
    """
    final_df = pd.concat(sorted_dfs, axis=1, join='inner')
    return final_df

In [186]:
def fill_nans(final_df):
    """ Fill the entries corresponding to the missed tests (which are represented as NaN values in the dataframe)
    
    Args:
        final_df (dataframe): returned by the get_final_dataframe(clean_dfs) function
        
    Returns:
        final_df (dataframe): all the NaN values filled with "M" (for missed tests)
    """
    for col in final_df.columns:
        final_df[col].fillna("M", inplace = True)
    return final_df

In [187]:
def generate_excel(file_names, section):
    """ Generates the xlsx files by calling the functions that are created above
    
    Args:
        file_names(list): has the names of all the csv files (that look like term test *.csv) from which all section names will
                          be taken out
        section (str): section name for which you want the dataframes and then the final excel files    
    """
    xls_file = f'1500 {section}.xlsx'
    list_of_dfs = remove_unwanted_cols(file_names)
    clean_dfs = section_only_results(list_of_dfs, section)
    sorted_dfs = sort_dfs(clean_dfs)
    final_df = get_final_dataframe(sorted_dfs)
    final_df = fill_nans(final_df)
    final_df.to_excel(xls_file)

In [188]:
def main():
    """
    The caller function that calls all the above functions. Also asks the user for number of files in the group to be analyzed. 
    """
    num_files = input("Enter the number of csv files in the group: ")
    file_names = [f"term test {str(i)}.csv" for i in range(1,int(num_files)+1)]
    sections = get_sections(file_names)
    for section in sections:
        generate_excel(file_names, section)

In [189]:
main()

Enter the number of csv files in the group: 9
